분류 기출
3. 정시 배송 여부 판단(2회 기출)

In [35]:
##0. 문제 의도 파악
#이커머스 정보 토대로 정시 배송 여부 분류(0:미배송,1:정시)

##1. 패키지 불러오기
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

##2. 파일 읽기
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv')

##3. 전처리
# print(x_train.info())
# print(x_train.describe())

#범주형 변수 값 확인: Warehouse_block, Mode_of_Shipment, Customer_care_calls, Product_importance, Gender
# print(x_train['Warehouse_block'].unique())
# print(x_train['Mode_of_Shipment'].unique())
# print(x_train['Customer_care_calls'].unique())
# print(x_train['Product_importance'].unique())
# print(x_train['Gender'].unique())

#Gender: 수동인코딩
x_train['Gender'] = x_train['Gender'].replace('M',0).replace('F',1)
x_test['Gender'] = x_test['Gender'].replace('M',0).replace('F',1)

#Customer_care_calls: 값 바꿔주고 int로 변환
x_train['Customer_care_calls'] = x_train['Customer_care_calls'].replace('$7',7).astype('int64')
x_test['Customer_care_calls'] = x_test['Customer_care_calls'].replace('$7',7).astype('int64')

#불필요한 칼럼 삭제: ID
x_test_id = x_test['ID']
x_train.drop(columns= 'ID', inplace=True)
x_test.drop(columns= 'ID', inplace=True)
y_train.drop(columns= 'ID', inplace=True)
y_test.drop(columns= 'ID', inplace=True)

#스케일링
scaler = MinMaxScaler()
x_train['Cost_of_the_Product'] = scaler.fit_transform(x_train[['Cost_of_the_Product']])
x_test['Cost_of_the_Product'] = scaler.transform(x_test[['Cost_of_the_Product']])
x_train['Weight_in_gms'] = scaler.fit_transform(x_train[['Weight_in_gms']])
x_test['Weight_in_gms'] = scaler.transform(x_test[['Weight_in_gms']])

#원핫인코딩: Warehouse_block, Mode_of_Shipment, Product_importance
x_dummy = pd.get_dummies(pd.concat([x_train, x_test], axis=0), drop_first=True)
len_split = x_train.shape[0]
x_train = x_dummy[:len_split]
x_test = x_dummy[len_split:]

#상관관계 파악
data = pd.concat([x_train, y_train], axis=1)
# print(data.corr())

##4. 데이터 분리
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

##5. 모델 생성 및 하이퍼 파라미터
model_dt = DecisionTreeClassifier(max_depth=10, random_state=42)
model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy', random_state=42)
model_xgb = XGBClassifier(n_estimators=100, max_depth=10, eval_metric='error', random_state=42)
model_svc = SVC(random_state=42)

##6. 모델 학습 및 평가, 앙상블
model_dt.fit(X_TRAIN, Y_TRAIN)
model_rf.fit(X_TRAIN, Y_TRAIN)
model_xgb.fit(X_TRAIN, Y_TRAIN)
model_svc.fit(X_TRAIN, Y_TRAIN)

pred_dt = model_dt.predict(X_TEST)
pred_rf = model_rf.predict(X_TEST)
pred_xgb = model_xgb.predict(X_TEST)
pred_svc = model_svc.predict(X_TEST)

print(roc_auc_score(Y_TEST, pred_dt))
print(roc_auc_score(Y_TEST,pred_rf))
print(roc_auc_score(Y_TEST, pred_xgb))
print(roc_auc_score(Y_TEST, pred_svc))

best_models = [('dt', model_dt), ('rf', model_rf)]
voting_clf = VotingClassifier(estimators=best_models, voting='soft')
voting_clf.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = voting_clf.predict(X_TEST)
print(roc_auc_score(Y_TEST, Y_TEST_PRED))

##7. 결과 예측
y_test_pred = pd.DataFrame(model_rf.predict(x_test)).rename(columns= {0:'Reached.on.Time_Y.N'})
print(roc_auc_score(y_test, y_test_pred))

##8. 파일 제출
final = pd.concat([x_test_id, y_test_pred], axis=1)
final.to_csv('/content/drive/MyDrive/bigData-main/004001009분류기출3.csv', index=False)

final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009분류기출3.csv')
print(final)

0.6833998419012433
0.6945935720596379
0.6504918393141726
0.6948186725622566
0.6949824780923314
0.6918547997811699
         ID  Reached.on.Time_Y.N
0      6811                    1
1      4320                    0
2      5732                    0
3      7429                    0
4      2191                    1
...     ...                  ...
4396   2610                    1
4397   3406                    0
4398  10395                    1
4399   3646                    0
4400    573                    1

[4401 rows x 2 columns]


분류기출
2. 이직여부 판단

In [93]:
##0. 문제 의도 파악
#사원정보 통해 이직여부 분류(0:노이직, 1:이직)

##1. 패키지 불러오기
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import mode

##2. 파일 읽기
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_test.csv')
# print(x_train.head())
# print(x_train.shape)
# print(x_test.head())
# print(x_test.shape)
# print(y_train.head())
# print(y_train.shape)

##3. 전처리
# print(x_train.info())
# print(x_train.describe())
# print(x_train.isnull().sum())
# print(x_test.isnull().sum())

# for col in x_train.columns:
#   if x_train[col].dtype == 'object':
#     print(col, x_train[col].unique())

# print(x_train.info())

#불필요한 칼럼 삭제: enrollee_id
x_test_enrollee_id = x_test['enrollee_id']
x_train.drop(columns= ['enrollee_id', 'city', 'company_size', 'company_type'], inplace= True)
x_test.drop(columns= ['enrollee_id', 'city', 'company_size', 'company_type'], inplace= True)
y_train.drop(columns= ['enrollee_id'], inplace= True)
y_test.drop(columns= ['enrollee_id'], inplace=True)

#결측치 처리
x_train['gender'].fillna(2, inplace=True)
x_test['gender'].fillna(2, inplace=True)

x_train['enrolled_university'].fillna('Other', inplace=True)
x_test['enrolled_university'].fillna('Other', inplace=True)

x_train['education_level'].fillna('Other', inplace=True)
x_test['education_level'].fillna('Other', inplace=True)

x_train['major_discipline'].fillna('Other', inplace=True)
x_test['major_discipline'].fillna('Other', inplace=True)

x_train['experience'].fillna(21, inplace=True)
x_test['experience'].fillna(21, inplace=True)
# print(mode(x_train['experience']))

x_train['last_new_job'].fillna(1, inplace=True)
x_test['last_new_job'].fillna(1, inplace=True)
# print(mode(x_train['last_new_job']))

#수동 인코딩: gender, relevent_experience
x_train['gender'] = x_train['gender'].replace('Male',0).replace('Female',1).replace('Other',2)
x_test['gender'] = x_test['gender'].replace('Male',0).replace('Female',1).replace('Other',2)

x_train['relevent_experience'] = x_train['relevent_experience'].replace('No relevent experience',0).replace('Has relevent experience',1)
x_test['relevent_experience'] = x_test['relevent_experience'].replace('No relevent experience',0).replace('Has relevent experience',1)

x_train['experience'] = x_train['experience'].replace('<1',0).replace('>20',21)
x_test['experience'] = x_test['experience'].replace('<1',0).replace('>20',21)

x_train['last_new_job'] = x_train['last_new_job'].replace('never',0).replace('>4',5)
x_test['last_new_job'] = x_test['last_new_job'].replace('never',0).replace('>4',5)

x_train['enrolled_university'] = x_train['enrolled_university'].replace('Other',0).replace('no_enrollment',1).replace('Part time course',2).replace('Full time course',3)
x_test['enrolled_university'] = x_test['enrolled_university'].replace('Other',0).replace('no_enrollment',1).replace('Part time course',2).replace('Full time course',3)

x_train['education_level'] = x_train['education_level'].replace('Other',0).replace('High School',1).replace('Graduate',2).replace('Primary School',3).replace('Masters',4).replace('Phd',5)
x_test['education_level'] = x_test['education_level'].replace('Other',0).replace('High School',1).replace('Graduate',2).replace('Primary School',3).replace('Masters',4).replace('Phd',5)
# print(x_train.info())

x_train['experience'] = x_train['experience'].astype('int64')
x_test['experience'] = x_test['experience'].astype('int64')
x_train['last_new_job'] = x_train['last_new_job'].astype('int64')
x_test['last_new_job'] = x_test['last_new_job'].astype('int64')
# print(x_train.info())

# for col in x_train.columns:
#   if x_train[col].dtype == 'object':
#     print(col, x_train[col].unique())

#라벨 인코딩: 과목
encoder = LabelEncoder()
x_train['major_discipline'] = encoder.fit_transform(x_train['major_discipline'])
x_test['major_discipline'] = encoder.fit_transform(x_test['major_discipline'])
# print(x_train.head())
# print(x_train.info())

#스케일링: experience, training_hours
# print(x_train.describe())
scaler = MinMaxScaler()
x_train['experience'] = scaler.fit_transform(x_train[['experience']])
x_test['experience'] = scaler.transform(x_test[['experience']])
x_train['training_hours'] = scaler.fit_transform(x_train[['training_hours']])
x_test['training_hours'] = scaler.transform(x_test[['training_hours']])

#상관관계 확인
# print(x_train.corr())

##4. 데이터 분리
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

##5. 모델 생성 및 하이퍼 파라미터
model_dt = DecisionTreeClassifier(criterion='entropy', max_depth= 10, random_state=42)
model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_xgb = XGBClassifier(n_estimators=100, max_depth=5, eval_metric='error', random_state=42)

##6. 모델 학습 및 평가, 앙상블
model_dt.fit(X_TRAIN, Y_TRAIN)
model_rf.fit(X_TRAIN, Y_TRAIN)
model_xgb.fit(X_TRAIN, Y_TRAIN)

pred_dt = model_dt.predict(X_TEST)
pred_rf = model_rf.predict(X_TEST)
pred_xgb = model_xgb.predict(X_TEST)

print(roc_auc_score(Y_TEST, pred_dt))
print(roc_auc_score(Y_TEST, pred_rf))
print(roc_auc_score(Y_TEST, pred_xgb))

# best_models = [('dt', model_dt), ('rf',model_rf), ('xgb', model_xgb)]
# voting_clf = VotingClassifier(estimators=best_models, voting='soft')
# voting_clf.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = model_dt.predict(X_TEST)
print(roc_auc_score(Y_TEST, Y_TEST_PRED))

#for문으로 최적의 random_state값 찾기
# roc_list = []
# for i in range(1,20):
#   X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state= 42)
#   model_dt = DecisionTreeClassifier(criterion='entropy', max_depth= i, random_state= 42)
#   model_rf = RandomForestClassifier(n_estimators=100, max_depth=i, random_state= 42)
#   model_xgb = XGBClassifier(n_estimators=100, max_depth=i, eval_metric='error', random_state=42)

#   model_dt.fit(X_TRAIN, Y_TRAIN)
#   model_rf.fit(X_TRAIN, Y_TRAIN)
#   model_xgb.fit(X_TRAIN, Y_TRAIN)

#   pred_dt = model_dt.predict(X_TEST)
#   pred_rf = model_rf.predict(X_TEST)
#   pred_xgb = model_xgb.predict(X_TEST)

#   roc_dt = roc_auc_score(Y_TEST, pred_dt)
#   roc_rf = roc_auc_score(Y_TEST, pred_rf)
#   roc_xgb = roc_auc_score(Y_TEST, pred_xgb)

#   roc_list.append((i, roc_dt, roc_rf, roc_xgb))

# roc_list.sort()
# print(roc_list[0])

##7. 결과 예측: target
y_test_pred = pd.DataFrame(model_dt.predict(x_test)).rename(columns= {0:'target'})
print(roc_auc_score(y_test, y_test_pred))

##8. 파일 제출
final = pd.concat([x_test_enrollee_id, y_test_pred], axis=1)
final.to_csv('/content/drive/MyDrive/bigData-main/004001009분류2.csv', index=False)
final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009분류2.csv')
# print(final)

0.6498202513617177
0.6446702571846953
0.6480565573410566
0.6498202513617177
0.6586340872497638
      enrollee_id  target
0            7129     0.0
1           31037     0.0
2           22179     0.0
3           29724     1.0
4           17977     0.0
...           ...     ...
6701         3601     0.0
6702         2745     0.0
6703        18520     0.0
6704        10067     0.0
6705         8203     1.0

[6706 rows x 2 columns]


In [99]:
##0. 문제 의도 파악
#사원정보 통해 이직여부 분류(0:노이직, 1:이직)

##1. 패키지 불러오기
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import mode

##2. 파일 읽기
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_test.csv')

##3. 전처리
#불필요한 칼럼 삭제: enrollee_id
x_test_enrollee_id = x_test['enrollee_id']
x_train.drop(columns= ['enrollee_id', 'city', 'company_size', 'company_type'], inplace= True)
x_test.drop(columns= ['enrollee_id', 'city', 'company_size', 'company_type'], inplace= True)
y_train.drop(columns= ['enrollee_id'], inplace= True)
y_test.drop(columns= ['enrollee_id'], inplace=True)

#결측치 처리
x_train['gender'].fillna(2, inplace=True)
x_test['gender'].fillna(2, inplace=True)

x_train['enrolled_university'].fillna('Other', inplace=True)
x_test['enrolled_university'].fillna('Other', inplace=True)

x_train['education_level'].fillna('other', inplace=True)
x_test['education_level'].fillna('other', inplace=True)

x_train['major_discipline'].fillna('Other', inplace=True)
x_test['major_discipline'].fillna('Other', inplace=True)

x_train['experience'].fillna(21, inplace=True)
x_test['experience'].fillna(21, inplace=True)
# print(mode(x_train['experience']))

x_train['last_new_job'].fillna(1, inplace=True)
x_test['last_new_job'].fillna(1, inplace=True)
# print(mode(x_train['last_new_job']))

#수동 인코딩
x_train['experience'] = x_train['experience'].replace('<1',0).replace('>20',21)
x_test['experience'] = x_test['experience'].replace('<1',0).replace('>20',21)

#스케일링: experience, training_hours
# print(x_train.describe())
scaler = MinMaxScaler()
x_train['experience'] = scaler.fit_transform(x_train[['experience']])
x_test['experience'] = scaler.transform(x_test[['experience']])
x_train['training_hours'] = scaler.fit_transform(x_train[['training_hours']])
x_test['training_hours'] = scaler.transform(x_test[['training_hours']])

#수동 인코딩: gender, relevent_experience
x_dummy = pd.get_dummies(pd.concat([x_train, x_test], axis=0))
len_split = x_train.shape[0]
x_train = x_dummy[:len_split]
x_test = x_dummy[len_split:]

#상관관계 확인
# print(x_train.corr())

##4. 데이터 분리
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

##5. 모델 생성 및 하이퍼 파라미터
model_dt = DecisionTreeClassifier(criterion='entropy', max_depth= 10, random_state=42)
model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_xgb = XGBClassifier(n_estimators=100, max_depth=5, eval_metric='error', random_state=42)

##6. 모델 학습 및 평가, 앙상블
model_dt.fit(X_TRAIN, Y_TRAIN)
model_rf.fit(X_TRAIN, Y_TRAIN)
# model_xgb.fit(X_TRAIN, Y_TRAIN)

pred_dt = model_dt.predict(X_TEST)
pred_rf = model_rf.predict(X_TEST)
# pred_xgb = model_xgb.predict(X_TEST)

print(roc_auc_score(Y_TEST, pred_dt))
print(roc_auc_score(Y_TEST, pred_rf))
# print(roc_auc_score(Y_TEST, pred_xgb))

# best_models = [('dt', model_dt), ('rf',model_rf), ('xgb', model_xgb)]
# voting_clf = VotingClassifier(estimators=best_models, voting='soft')
# voting_clf.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = model_dt.predict(X_TEST)
# print(roc_auc_score(Y_TEST, Y_TEST_PRED))

##7. 결과 예측: target
y_test_pred = pd.DataFrame(model_dt.predict(x_test)).rename(columns= {0:'target'})
# print(roc_auc_score(y_test, y_test_pred))

##8. 파일 제출
final = pd.concat([x_test_enrollee_id, y_test_pred], axis=1)
final.to_csv('/content/drive/MyDrive/bigData-main/004001009분류2.csv', index=False)
final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009분류2.csv')
# print(final)

0.6279119183943092
0.6252541586560949


분류 기출
11. 당뇨여부판단

In [118]:
##0. 문제의도 파악
#환자 정보 통해 '당뇨여부' 분류(0:정상, 1:당뇨)

##1. 패키지 불러오기
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

##2. 파일 읽기
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/x_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/x_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/y_train.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/y_test.csv')
# print(x_train.head())
# print(x_train.shape)
# print(x_test.head())
# print(x_test.shape)
# print(y_train.head())
# print(y_train.shape)

##3. 전처리
# print(x_train.info())
# print(x_train.describe())

#불필요한 칼럼 삭제: ID
x_test_id = x_test['ID']
x_train.drop(columns= 'ID', inplace=True)
x_test.drop(columns= 'ID', inplace=True)
y_train.drop(columns= 'ID', inplace=True)
y_test.drop(columns= 'ID', inplace=True)

#데이터 스케일링
scaler = MinMaxScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns= x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns= x_train.columns)

#상관관계 파악
# print(x_train.corr())

##4. 데이터 분리
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

##5. 모델 생성 및 하이퍼 파라미터
model_dt = DecisionTreeClassifier(max_depth=10, random_state=42)
model_rf = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=10, random_state=42)
model_xgb = XGBClassifier(max_depth=10, eval_metric='error', random_state=42)

##6. 모델 학습 및 평가, 앙상블
model_dt.fit(X_TRAIN, Y_TRAIN)
model_rf.fit(X_TRAIN, Y_TRAIN)
model_xgb.fit(X_TRAIN, Y_TRAIN)

pred_dt = model_dt.predict(X_TEST)
pred_rf = model_rf.predict(X_TEST)
pred_xgb = model_xgb.predict(X_TEST)

print(roc_auc_score(Y_TEST, pred_dt))
print(roc_auc_score(Y_TEST, pred_rf))
print(roc_auc_score(Y_TEST, pred_xgb))

best_models = [('rf', model_rf), ('xgb', model_xgb)]
voting_clf = VotingClassifier(estimators=best_models, voting='soft')
voting_clf.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = voting_clf.predict(X_TEST)
print(roc_auc_score(Y_TEST, Y_TEST_PRED))

##7. 결과 예측: outcome
y_test_pred = pd.DataFrame(voting_clf.predict(x_test)).rename(columns={0:'Outcome'})
print(roc_auc_score(y_test, y_test_pred))
# print(y_test_pred)

##8. 파일 제출
final = pd.concat([x_test_id, y_test_pred], axis=1)
final.to_csv('/content/drive/MyDrive/bigData-main/004001009분류11.csv', index=False)
# final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009분류11.csv')
# print(final)

0.6265734265734265
0.7325174825174825
0.7402097902097903
0.7479020979020979
0.8050505050505049
     Outcome
0          0
1          0
2          0
3          1
4          0
..       ...
149        1
150        1
151        0
152        1
153        1

[154 rows x 1 columns]
      ID  Outcome
0     13        0
1     18        0
2     29        0
3     33        1
4     34        0
..   ...      ...
149  751        1
150  752        1
151  759        0
152  765        1
153  767        1

[154 rows x 2 columns]


교재 - 작업형1
1. Top 10 구하기

In [122]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#오름차순 정렬
df.sort_values(by='MEDV', ascending=True, inplace= True)
result = df['MEDV'].head(10)
print(result)

398    5.0
405    5.0
400    5.6
399    6.3
414    7.0
489    7.0
401    7.2
385    7.2
415    7.2
387    7.4
Name: MEDV, dtype: float64


2. 결측치 확인하기

In [133]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#결측치 확인
# print(df.head())
# print(df.isnull().sum())

#평균값 대치
df_mean = df['RM'].copy()
rm_mean = df['RM'].mean()
df_mean.fillna(rm_mean, inplace=True)

#결측치 삭제
df_del = df['RM'].copy()
df_del.dropna(inplace=True)

#표준편차 값 차이(절대값)
result = abs(df_mean.std() - df_del.std())
print(result)

0.010595546094104513


3. 이상값 확인하기

In [136]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#평균값, 표준편차
zn_mean = df['ZN'].mean()
zn_std = df['ZN'].std()

#최소/최대경계값
zn_min = zn_mean - (1.5 * zn_std)
zn_max = zn_mean + (1.5 * zn_std)

#이상값 구하기
df_min = df[df['ZN'] < zn_min]['ZN']
df_max = df[df['ZN'] > zn_max]['ZN']

#이상값 합계
result = df_max.sum()
print(result)

3462.5


4. 사분위수 구하기

In [141]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#칼럼 제외
data = df.copy()
data.drop(columns= ['CHAS', 'RAD'], inplace=True)

#IQR 구하기
data_desc = data.describe()
data_iqr = data_desc.iloc[[4,6]].T

data_iqr['IQR'] = data_iqr['75%'] - data_iqr['25%']
result = data_iqr['IQR']
print(result)

CRIM         3.595038
ZN          12.500000
INDUS       12.910000
NOX          0.175000
RM           0.736000
AGE         49.050000
DIS          3.088250
TAX        387.000000
PTRATIO      2.800000
B           20.847500
LSTAT       10.005000
MEDV         7.975000
Name: IQR, dtype: float64


5. 순위 구하기

In [145]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#내림차순 정렬
df = data.sort_values(by='MEDV', ascending=False)

#30번째로 큰 값으로 대치(41.7)
df['MEDV'].iloc[0:29] = 41.7
# print(df['MEDV'].head(30))

#평균값, 중위값, 최솟값, 최댓값
print(df['MEDV'].mean(), df['MEDV'].median(), df['MEDV'].min(), df['MEDV'].max())

22.1796442687747 21.2 5.0 41.7


6. 그룹별 집계/요약하기

In [157]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')
# print(df.head())

#중위값 보다 큰 데이터 확인
tax_median = df['TAX'].median()
df_new = df[df['TAX'] > tax_median]

#그룹화, 각 그룹의 데이터 개수 & COUNT 칼럼 출력
result = pd.DataFrame(df_new.groupby(['CHAS', 'RAD'])['RAD'].count()).rename(columns= {'RAD':'COUNT'})
print(result)

          COUNT
CHAS RAD       
0    1        3
     2        2
     3        5
     4       33
     5       51
     6       17
     24     124
1    5        7
     24       8


7. 오름차순/내림차순 정렬하기

In [165]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#TAX칼럼 오름차순 정렬
df_asc = df['TAX'].copy()
df_asc.sort_values(ascending= True, inplace=True)

#내림차순 정렬
df_desc = df['TAX'].copy()
df_desc.sort_values(ascending= False, inplace= True)

#인덱스 재설정
df_asc.reset_index(drop=True, inplace=True)
df_desc.reset_index(drop=True, inplace=True)

df_concat = pd.concat([df_asc, df_desc], axis=1)

#(오름차순 - 내림차순)의 분산 값
df_concat['diff'] = df_concat.iloc[:,1] - df_concat.iloc[:,0]
print(df_concat['diff'].var())

101954.72475247525


In [171]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#오름차순 정렬
df_asc = df['TAX'].copy()
df_asc.sort_values(ascending=True, inplace=True)

#내림차순 정렬
df_desc = df['TAX'].copy()
df_desc.sort_values(ascending=False, inplace=True)

#인덱스 재설정
df_asc.reset_index(drop=True, inplace=True)
df_desc.reset_index(drop=True, inplace=True)

df_concat = pd.concat([df_asc, df_desc], axis=1)

#(오름차순 - 내림차순) 값 차이 분산 값
diff = abs(df_concat.iloc[:,0] - df_concat.iloc[:,1])
print(diff.var())

28490.598645951555


8. 최소최대 변환하기

In [178]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#최소최대 척도 변환
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns= df.columns)

#0.5보다 큰 값 가지는 레코드 수
result = df[df['MEDV'] > 0.5]['MEDV'].count()
print(result)

106


9. 빈도값 구하기

In [185]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#반올림
df = round(data['AGE'].copy())

#AGE칼럼 최빈값과 그 개수
from scipy.stats import mode
print(int(mode(df)[0]), int(mode(df)[1]))

100 43


10. 표준 변환하기

In [187]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#표준화 척도 변환
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns= df.columns)

#0.4보다 크면서 0.6보다 작은 값들에 대한 평균
result = df[(df['DIS'] > 0.4) & (df['DIS'] < 0.6)]['DIS'].mean()

#반올림
print(round(result, 2))

0.48


11. 유니크한 값 구하기

In [195]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#칼럼 개수
df_col = df.columns
df_col_size = len(df_col)

#칼럼 별 유니크 값 개수 구하기
sum = 0
for col in df_col:
  sum += int(pd.DataFrame(df[col].unique()).count())

#평균 = 전체 칼럼별 유니크한 값 개수 / 칼럼 개수
print(sum/df_col_size)

218.0


In [196]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/boston.csv')

#칼럼 목록
df_col = df.columns

#칼럼별 유니크한 값 개수 합
sum=0
for col in df_col:
  sum += int(pd.DataFrame(df[col].unique()).count())

#평균값
print(sum / df_col.size)

218.0
